# 4、情感计算更新

In [13]:
import pickle
from jieba import analyse
import pickle
import os
import jieba
from jieba import analyse
from snownlp import SnowNLP
from snownlp.sentiment import Sentiment
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from elasticsearch import Elasticsearch
import pandas as pd

In [14]:
class esWeiboTweetRetrieval():
    '''
    根据论文检索需求进行功能的微调
    '''
    _instance = None
    _first_init = True
    def __new__(cls, *args, **kw):
        if not cls._instance:
            cls._instance = super(esWeiboTweetRetrieval, cls).__new__(cls)  
        return cls._instance
    
    def __init__(self, host, port):
        '''
        使用ES进行论文检索 指定host、port以及专利index之后进行检索
        '''
        super(esWeiboTweetRetrieval, self).__init__()
        self.es = Elasticsearch(hosts=host, port=port, timeout=30, max_retries=10, retry_on_timeout=True)
        self.indexName = 'weibo-tweet-index'

    def do_search(self, W_search_query, W_content, volume):
        '''
        do_search方法执行具体检索过程
        wordQuery 本应为查询对应微博时所用的检索词 此处暂时不用 目前暂时只检索微博评论数据后续再修改对象为
        
        volume为每次检索返回的数目
        '''
        queryBody = {
        "query": {
        "bool": {
        "must": [ ],
        "must_not": [ ],
        "should": [
        {
        "query_string": {
        "default_field": "W_content",
        "query": W_content
        }
        }
        ,
        {
        "query_string": {
        "default_field": "W_search_query",
        "query": W_search_query
        }
        }
        ]
        }
        },
        "from": 0,
        "size": volume,
        "sort": [ ],
        "aggs": { }
        }
        result = self.es.search(index=self.indexName, body=queryBody)
        return result

    def format_search(self, result):
        '''
        format_search方法对检索结果进行格式化 构建符合要求的字段进行返回
        输入result为检索结果 提取其中的检索结果进行后处理
        使用ES检索后得到的结果中result['hits']['hits']为数组格式数据
        其中每一个元素为一个dict 对应部分字段
        '''
        docs = result['hits']['hits']
        docs = [i['_source'] for i in docs]
        targetKeyList = 'W_ID,W_weibo_id,W_search_query,W_nick_name,W_weibo_url,W_created_at,\
        W_like_num,W_repost_num,W_comment_num,W_content,W_user_id,W_image_url,W_video_url,W_tool,W_location,\
        W_location_map_info,W_origin_weibo_url,W_origin_weibo_content,W_crawl_time'
        targetKeyList = [i.strip() for i in targetKeyList.split(',')]
        dict_filter_by_keys = lambda d: {k: d[k] for k in targetKeyList}
        dict_filter_id = lambda d: {k if not k == '_id' else 'id': d[k] for k in d}
        docs = list(dict_filter_by_keys(doc) for doc in docs)
        return docs
    def Retrieval(self, W_search_query, W_content, volume):
        result = self.do_search(W_search_query, W_content, volume)
        docs = self.format_search(result)
        return docs

In [17]:
# input： 微博正文检索结果
# output： {
#         dict_emo         微博情感七分类得分
#         content_emo      微博情感七分类对应各5条微博
#         dict_senti          微博情感二分类得分
#         content_senti   负向微博前5条  及 正向微博前5条
# }
def get_weibo_senti(result):
    conment_list = lambda x:x[:5] if len(x)>5 else x
    happy = ["PA","PE"]#乐
    like = ["PD","PH","PG","PB","PK"]#好
    anger = ["NA"]#怒
    sad = ["NB","NJ","NH","PF"]#哀
    afraid = ["NI","NC","NG"]#惧
    boring = ["NE","ND","NN","NK","NL"]#恶
    suprise = ["PC"]#惊
    emotion = pd.read_excel('../data/weibo_data/senti onthology.xlsx', header=0, encoding='utf8')
    words = emotion["词语"].tolist()
    emo = emotion["情感分类"].tolist()
    core = emotion["强度"].tolist()
    pos_neg = emotion["极性"].tolist()
    res = [i['W_content'] for i in result]
    num_happy=0
    num_like =0
    num_anger=0
    num_sad = 0
    num_afraid = 0
    num_boring = 0
    num_suprise = 0
    num_pos=0
    num_neg=0
    happy_content = []
    like_content = []
    anger_content = []
    sad_content = []
    afraid_content = []
    boring_content = []
    suprise_content = []
    pos_content = []
    neg_content = []
    if len(res)>500:
        N=500
    else:
        N = len(res)
    for i in res[:N]: 
        sore_happy=0
        sore_like =0
        sore_anger=0
        sore_sad = 0
        sore_afraid = 0
        sore_boring = 0
        sore_suprise = 0
        sore_pos=0
        sore_neg=0
        if len(i)>4:
            for word in jieba.lcut(i):
                if word in emotion["词语"].tolist():
                    aa = emotion.loc[emotion['词语']==word]
                    if aa["情感分类"].all() in happy:                  
                         sore_happy= aa["强度"].tolist()[0]+sore_happy
                    elif aa["情感分类"].all() in like:
                        sore_like = aa["强度"].tolist()[0]+sore_like
                    elif aa["情感分类"].all() in anger:
                        sore_anger = aa["强度"].tolist()[0]+sore_anger
                    elif aa["情感分类"].all() in sad:
                        sore_sad = aa["强度"].tolist()[0]+sore_sad
                    elif aa["情感分类"].all() in afraid:
                        sore_afraid = aa["强度"].tolist()[0]+sore_afraid
                    elif aa["情感分类"].all() in boring:
                        sore_boring = aa["强度"].tolist()[0]+sore_boring
                    elif aa["情感分类"].all() in suprise:
                        sore_suprise = aa["强度"].tolist()[0]+sore_suprise
                    if aa["极性"].tolist()[0] == 1:
                        sore_pos = aa["强度"].tolist()[0]+sore_pos
                    elif aa["极性"].tolist()[0] == 2:
                        sore_neg = aa["强度"].tolist()[0]+sore_neg
            emo=[]
            emo.append(sore_happy)
            emo.append(sore_like)
            emo.append(sore_anger)
            emo.append(sore_sad)
            emo.append(sore_afraid)
            emo.append(sore_boring)
            emo.append(sore_suprise)
            senti = []
            senti.append(sore_pos)
            senti.append(sore_neg)
            if max(emo) == sore_happy:
                
                num_happy = num_happy+1
                if sore_happy>10:
                    happy_content.append(i)
            elif max(emo) == sore_like:
                if sore_like>15:
                    like_content.append(i)
                num_like = num_like+1
            elif max(emo) == sore_anger:
               
                anger_content.append(i)
                num_anger = num_anger+1
            elif max(emo) == sore_sad:
                
                sad_content.append(i)
                num_sad = num_sad+1
            elif max(emo) == sore_afraid:
                afraid_content.append(i)
                num_afraid = num_afraid+1
            elif max(emo) == sore_boring:

                boring_content.append(i)
                num_boring = num_boring+1
            elif max(emo) == sore_suprise:

                suprise_content.append(i)
                num_suprise = num_suprise+1
            if max(senti) == sore_pos:
                if sore_pos>14:
                    pos_content.append(i)
                num_pos = num_pos+1
            else:
                if sore_neg>14:
                    neg_content.append(i)
                num_neg = num_neg+1
            sum1 = num_happy+num_like+num_anger+num_sad+num_afraid+num_boring+num_suprise
            dict_emo = {"happy":round(num_happy/sum1,3),"like":round(num_like/sum1,3),"anger":round(num_anger/sum1,3),
                        "sad":round(num_sad/sum1,3),"afraid":round(num_afraid/sum1,3),"boring":round(num_boring/sum1,3),
                        "suprise":round(num_suprise/sum1,3)}
            dict_senti = {"positive":round(num_pos/(num_pos+num_neg),3),"negtive":round(num_neg/(num_pos+num_neg),3)}
            content_emo = {"happy":conment_list(happy_content),"like":conment_list(like_content),"anger":conment_list(anger_content),
                        "sad":conment_list(sad_content),"afraid":conment_list(afraid_content),"boring":conment_list(boring_content),
                        "suprise":conment_list(suprise_content)}
            content_senti = {"positive":conment_list(pos_content),"negtive":conment_list(neg_content)}
    return dict_emo,content_emo,dict_senti,content_senti

In [32]:
esObj = esWeiboTweetRetrieval(host='10.8.128.205',port=49200)
docs = esObj.Retrieval(W_search_query='垃圾分类',W_content='垃圾分类',volume=100)
dict_emo,content_emo,dict_senti,content_senti = get_weibo_senti(docs)

make_dict = lambda x:[{'name':k,'value':x[k]} for k in x]

dict_senti = make_dict(dict_senti)
dict_emo = make_dict(dict_emo)

result = {'dict_senti':dict_senti,
         'dict_emo':dict_emo,
         'content_senti':content_senti,
         'content_emo':content_emo}

In [36]:
result

{'dict_senti': [{'name': 'positive', 'value': 0.85},
  {'name': 'negtive', 'value': 0.15}],
 'dict_emo': [{'name': 'happy', 'value': 0.52},
  {'name': 'like', 'value': 0.3},
  {'name': 'anger', 'value': 0.0},
  {'name': 'sad', 'value': 0.0},
  {'name': 'afraid', 'value': 0.16},
  {'name': 'boring', 'value': 0.02},
  {'name': 'suprise', 'value': 0.0}],
 'content_senti': {'positive': ['#垃圾分类# 【农村垃圾分类进行时】永宁街道召开垃圾分类工作再推进会议，要求各社区和相关部门整体推进垃圾分类工作、完善垃圾分类各类设施、加大垃圾分类宣传力度，重点落实后期居民垃圾分类工作的运行。',
   '#垃圾分类# 【图解：谁是真正的“垃圾之王”？】受到垃圾分类在上海的落地实施，以及生活垃圾分类入法等消息刺激，A股催生出了全新的概念板块：垃圾分类板块。6月25日早盘，垃圾分类概念迅速拉升。然而，谁才是垃圾分类中真正的王牌企业？中国"科"公司评选之垃圾分类板块:谁才是"垃圾之王"?',
   '#垃圾分类# 【图解：谁是真正的“垃圾之王”？】受到垃圾分类在上海的落地实施，以及生活垃圾分类入法等消息刺激，A股催生出了全新的概念板块：垃圾分类板块。6月25日早盘，垃圾分类概念迅速拉升。然而，谁才是垃圾分类中真正的王牌企业？中国"科"公司评选之垃圾分类板块:谁才是"垃圾之王"?',
   '#垃圾分类# 为进一步关普及垃圾分类的理念和知识，近期，市城治办开展了垃圾分类进机关的宣讲活动。此次活动通过现场垃圾分类知识的讲解，介绍垃圾分类的方法和必要性，就生活垃圾分类、厨余垃圾就地减量等方面介绍了一些成功经验，将垃圾分类工作结合到日常的工作中去，真正让垃圾分类理念入心入脑，为我市垃圾分类工作和为城市生活垃圾减量化、资源化、无害化打下坚实基础。[组图共2张]',
   '#垃圾分类# 今天，张江镇继续拓展垃圾分类宣传